loading required libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 5000)
from dotenv import load_dotenv
import json
from src.pdf import create_pdf
from src.gpt import evaluate_gpt
from src.utils import (
    load_preprocess_data,
    extract_all_diseases_per_department,
    filterDepartment,
    getDepartmentStatistics,
    run_prediction,
    evaluate_results)
load_dotenv()
from prompts import single_shot_disease_only_prompt,open_ended__top4_prompt,with_reasons_prompt

importing the dataset

In [ ]:
filePath="dataset/clinicallab/data_en.json"
df=load_preprocess_data(filePath)


In [ ]:
def extract_disease_names(diagnoses):
    disease_names = []
    for diagnosis in diagnoses:
        disease_name = diagnosis.split(':')[0]  # Split and get the disease name
        if len(disease_name) < 40:  # Check if the name is less than 40 characters
            disease_names.append(disease_name)
    return disease_names

# Apply the function to filter and transform the 'differential-diagnosis' field
df['differential_diagnosis'] = df['differential_diagnosis'].apply(extract_disease_names)

# Filter rows where the number of elements in 'differential-diagnosis' is greater than 1
filtered_df = df[df['differential_diagnosis'].apply(lambda x: len(x) > 1)]
filtered_df['differential_diagnosis'] = filtered_df['differential_diagnosis'] + filtered_df['principal_diagnosis'].apply(lambda x: [x] if x else [])

In [ ]:
print(len(df))
# filtered_df = df[df['differential_diagnosis'].apply(lambda x: len(x) > 2)]
print(len(filtered_df))

In [ ]:
filtered_df.clinical_department.value_counts().sort_values(ascending=False)

In [ ]:
departments=["nephrology department",
             "gynecology department",
            "endocrinology department", 
            "neurology department",
             "pediatrics department",
             "cardiac surgical department",                          
             "gastrointestinal surgical department",
             "respiratory medicine department",
             "gastroenterology department",
             "urinary surgical department",
             "hepatobiliary and pancreas surgical department",
             "hematology department"
            ]
models = ["gpt-4","llama3.1","gemma2", "phi3:14b", "mistral-nemo"]

In [ ]:
filtered_df = filtered_df[filtered_df['clinical_department'].isin(departments)]
print(len(filtered_df))

In [ ]:
729/3

filtering the dataset

In [ ]:


# type="with_reasons"
# prompt=with_reasons_prompt
type="single_shot"
prompt=single_shot_disease_only_prompt
# type="open_ended"
# # prompt=open_ended__top4_prompt# 
# run_prediction(df,prompt,departments,models=models,type=type,skip=5)

In [ ]:
# file_path="results/nephrology department_single_shot_without_sorted_result_findings.json"
# with open(file_path, "r") as file:
#     data = json.load(file)
# evaluate_results(data)

In [1]:

import os
import json
from src.utils import evaluate_department_results
types=["single_shot","with_reasons"]
for type in types:
    all_files=os.listdir("results")
    all_results={}
    for file in all_files:
        department_name=file.split("_")[0]
        print(department_name)
        if type in file:
            file_path=os.path.join("results",file)
            department_name=file.split("_")[0]
            with open(file_path, "r") as file:
                data = json.load(file)
                results=evaluate_department_results(data)
            all_results[department_name]=results
    with open(f"{type}.json", "w") as outfile: 
        json.dump(all_results, outfile)

urinary surgical department
gastrointestinal surgical department
['gpt-4o', 'llama3.1', 'gemma2']
metabolic syndrome ["Metabolic syndrome", True] 

metabolic syndrome ["Metabolic syndrome", True] 

metabolic syndrome ["Metabolic syndrome", True] 

metabolic syndrome ["Hypothalamic obesity", False] 

metabolic syndrome ["Hypothalamic obesity", False] 

metabolic syndrome ["Hypothalamic obesity", False] 

metabolic syndrome ["Metabolic syndrome", True] 

metabolic syndrome ["Metabolic syndrome", True] 

metabolic syndrome ["Metabolic syndrome", True] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Right ovarian cyst torsion", False] 

acute appendicitis ["Acute appendicitis", True] 

acute appendicitis ["Intestinal ob